# FAhQL 

History based Q-Learning

As per http://proceedings.mlr.press/v29/Daswani13.pdf

The idea is to extract features form this history, then estimate the Q values from these features directly (i.e. without learning a model)

Two methods for extracting features talked about in the paper.

* **Context trees** which provide a robust way to learn context dependant history lengths.

* **Event selector** which extracts a set of observations from specific times prior in history.

Once features have been extracted a linear function approximator is learned by mimizing squared Q-learning error.

We implement just the FAhQL algorithm here.

$\mathbf{\text{Cost}_{\text{QL}}(\xi)}$

We calculate the cost of a particiular event selector set $\xi$ as 

$$Cost_{QL}(\xi) := \min_w \frac{1}{2} \sum_{t=1}^n (r_{t+1} + \gamma \max_a \xi (h_{t+1},a_t)^T w) ^2 + \text{reg}(\xi)$$

That is, for the optimal $w$, how well did the function approximator approximate $Q$ values over the trajectory.
The regularization used is $\frac{\beta}{2}k\log_2(n)$ which for fix history size is simply the $l0$ norm, i.e. propotional to $k$ the number of features present in the map.

**Jumping Distribution**
In this case the jumping distribution is simply adding or removing a single feature.

**The Agent**

We start by giving the agent some initial history generated with random actions, and a starting map.

Epoch

1. Generate a new map, and either keep our current map or switch to the new one

2. Calculate $Q$ values based on states generated by our current map

3. Act according to the policy formed by $Q$ for m steps.

**Parameters used in paper**

100 Epochs
Each Epoch has 100 iterations
Stop $\epsilon$ exploration after 50 epochs
Run each experiment 30 times.
Number of annealing steps was 10 or 20 depending on algorithm / problem
An initial history of 200 time steps with random actions was used.


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import gym

from utils.utils import clip, smooth, int_to_bits, bits_to_int, softmax
from agents.agent import Agent

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# initial parameters

# todo: initialize these as per paper...

BETA = 1.0
GAMMA = 0.99

# our action space
action_space = []

# our observation space
observation_space = []

In [ ]:
def simulated_annealing(history, initial_map, cost, cooling_schedule, get_neighbour, N=10)
    """ 
        Find an improved map via simulated annealing.
        @param history The history so far, (observation, action, reward)
        @param initial_map An initial starting map
        @param cost A function mapping from (history, map) to the cost of that map
        @param schceule Temperature cooling schedule
        @param get_neighbour Function taking a map and generating a random neighbour
        @param n Number of simulated annealing iterations        
    """
    current_map = best_map = initial_map
    current_cost = best_cost = cost(h, initial_map)
    for t in range(N):
        candidate_map = get_neighbour(current_map)
        candidate_cost = cost(h, candidate_map)
        delta = current_cost - candiate_cost
        T = cooling_schedule(t)
        p = np.random.rand()
        if delta > 0 or p < np.exp(delta/T):
            current_map = candidate_map
            current_cost = candidate_cost
            if current_cost < best_cost:
                best_map = current_map
                best_cost = current_cost    
                
def get_neighbour(initial_map):
    """ Returns the neighbour of a event selector. """
    new_map = initial_map.copy()
    if np.random.randint(2) == 0:        
        """ Add an event. """
        random_observation = np.random.randint(len(observation_space))
        # the paper doesn't document how to select m, so I'm using poisson
        m = np.random.poisson(5)
        initial_map.append((m, random_observation))
    else:
        """ Remove an event. """
        del new_map[np.random.randint(len(new_map))]
    return new_map                                    

def cooling_schedule(t, start_T, cool_rate):
    """ Expoential cooling decay. """
    return start_T * np.exp(-cool_rate * t)
                           
def FAQ_learn(initial_w, states, rewards, actions):
    """
        Q learning via function approximation.
        This is learned off-policy with the supplied states rewards, and actions.
        If necessary the history is looped over.
                
        @param initial_w The initial weights for function approximation
        @param states State history list
        @param states Reward history list
        @param states Action history list
        
        @return The optimal weights for linear function approximation
    """
    
    #this update was taken from
    #https://openresearch-repository.anu.edu.au/bitstream/1885/110545/1/Daswani%20Thesis%202016.pdf
    
    w = initial_w
    
    for i in range(1000):
        
        state   = states[(i+1) % len(states)]
        reward  = reward[(i+1) % len(reward)]
        action  = action[(i+1) % len(action)]
        prev_state  = states[i % len(states)]
        prev_reward = reward[i % len(reward)]
        prev_action = action[i % len(action)]
                                    
        # in this case phi(s,a) is just s, but really we should be including a somehow...
        delta = reward + GAMMA * (w @ state) - (w @ prev_state)
        
        # todo add normalizating constant, sum_i phi_i(s,a)        
        w = w + alpha * delta * phi(state,action)
        
        prev_state = state
        prev_action = action        
    
    
def cost(phi, history):
    """ Cost of given map based on history. 
        @param phi map from history to state
        @param history list of (observation, action, reward) 
    """
    
    observations, actions, rewards = zip(*history)
    
    # generate our states via the mapping phi. 
    states = []
    for t in range(len(history)):
        states.append(phi(history[:t]))
    
    rewards = phi(event for event in history)
    
    # learn the best weights for our linear function approximation
    w = Q_learn(states, rewards, actions)
    
    # not sure how to include actions in the map phi, so I'll just ignore the action.
    # this ends up being a state value rather than a action-state value though?
    # really want we want is seperate weights for each action based on the features.
    xi = lambda history, action : phi(history)
    
    return cost_ql(w, xi, history)
    

def cost_ql(w, xi, history):
    """ Calculate the cost of the feature extractor xi, according to some history.
        @param w weights for linear function approximation (np array)
        @param xi function from (history,action) to feature vector.
        @param history list of (observation, reward, action)        
    """
    
    # length of feature vector
    k = length(w)
    
    # length of history
    n = length(history)
    
    regularization = BETA / 2 * k * log_2(n)
    
    cost_sum = 0
    for t in range(1,n):
        
        observation_prev, reward_prev, action_prev = history[t-1]
        observation, reward, action = history[t]
        
        action_value = [xi(history[:t+1], a) @ w for a in action_space]
                    
        td = (reward + GAMMA * max(action_value)) - xi(history[:t], action_prev) @ w
        
        cost_sum += 0.5*(td**2)
         
    return cost_sum + regularization
            

In [17]:
class FAhQL(Agent):
    """ Function approximation history based Q-Learning agent."""
    
    def __init__(self):
        pass
            
    def act(self, observation, reward):
        """ find optimal action based on observation and reward. """
        pass
    

TypeError: unsupported operand type(s) for @: 'list' and 'list'

**Notes:**
    
Daswani seems to be finding a map the mimizes the TD error, however according to http://papers.nips.cc/paper/8200-non-delusional-q-learning-and-value-iteration.pdf it is not a good idea to mimize the TD error, but rather the belman error.  Perhaps, in this case, as we are trying to find a good map, rather then solve it, this is still helpful?
        
**Questions:**
What is the difference between hQL and CT-MDP?

Why are we using $\frac{\beta}{2}k\log_2(n)$ as the regulizer.  Seems strange that the the cost of $\xi$ increases with the length of the history?  Shouldn't it be something like $\log_2(n^*)$ where $n^*$ is the longest time we look back in history?
* My thoughts on this is that the number of bits require for each feature selector index increases logarithimically with the length of the history.

Are there any guarintees that the event selector will generate a MDP, or do we not care.
* Talked to Daswani about this, said it won't be a MDP and refered me to Sultan's 2017 paper.

Seems like maybe the suffix tree defined here is much simpler than the context tree described by Nyguen.  No talk about dealing with MDP constrain though?

Why is the event selector observation only?  This would make the Maze4x4 problem unsolvable as it gives uninformative observations.

Cost algorithm: What is $\phi_{1:n}$

Cost algorithm: Shouldn't $\phi(h_t)$ be $\phi(h_{1:t})$?

Why is $\xi$ a map from history and action to a feature vector, not just history?
* Have a look at  https://danieltakeshi.github.io/2016/10/31/going-deeper-into-reinforcement-learning-understanding-q-learning-and-linear-function-approximation/, it explains this quite well.
